In [4]:
import redis

r = redis.Redis(host='redis', port=6379, decode_responses=True)

count_p = 0
for key in r.scan_iter("actor:*"):
    last_name = r.hget(key, "last_name")
    if last_name and last_name.strip().lower().startswith("p"):
        count_p += 1

print(f" Acteor start name start with 'P' : {count_p}")

selected_movies = []

for key in r.scan_iter("movie:*"):
    try:
        year = int(r.hget(key, "release_year"))
        votes = int(r.hget(key, "votes"))
        if year > 2010 and votes > 100000:
            title = r.hget(key, "title")
            selected_movies.append((title, year, votes))
    except:
        continue

print(f"\ Movies > 2010 with over 100 000 rev :")
for m in selected_movies:
    print(f"  - {m[0]} ({m[1]}), votes: {m[2]}")

best_per_genre = {}

for key in r.scan_iter("movie:*"):
    try:
        genre = r.hget(key, "genre")
        rating = float(r.hget(key, "rating"))
        title = r.hget(key, "title")
        if genre:
            if genre not in best_per_genre or rating > best_per_genre[genre][1]:
                best_per_genre[genre] = (title, rating)
    except:
        continue

for genre, (title, rating) in best_per_genre.items():
    redis_key = f"top_movies_by_genre:{genre}"
    r.hset(redis_key, mapping={"title": title, "rating": rating})
    print(f"best movie '{genre}' : {title} (note {rating}) -> stocké dans {redis_key}")


 Acteor start name start with 'P' : 64
\ Movies > 2010 with over 100 000 rev :
  - Straight Outta Compton (2015), votes: 127258
  - Spider-Man: Homecoming (2017), votes: 473931
  - Southpaw (2015), votes: 154514
  - Iron Man 3 (2013), votes: 726152
  - Noah (2014), votes: 203170
  - 1917 (2019), votes: 290813
  - The Revenant (2015), votes: 441653
  - Batman v Superman: Dawn of Justice (2016), votes: 415255
  - Zootopia (2016), votes: 216939
  - Steve Jobs (2015), votes: 102485
  - Spider-Man: Into the Spider-Verse (2018), votes: 302962
  - The Babadook (2014), votes: 185189
  - The Judge (2014), votes: 140247
  - Suicide Squad (2016), votes: 249658
  - Divergent (2014), votes: 412364
  - Hercules (2014), votes: 117904
  - Birdman or (The Unexpected Virtue of Ignorance) (2014), votes: 408054
  - A Million Ways to Die in the West (2014), votes: 138651
  - Predestination (2014), votes: 172521
  - The Interview (2014), votes: 251910
  - Spotlight (2015), votes: 220052
  - Captain America: